In [1]:
! pwd

/mnt/localdata/geng/lib/lmtc-eurlex57k


In [1]:
import json
import re
import time
import tempfile
import glob
import tqdm

import numpy as np
from copy import deepcopy
from collections import Counter
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import Sequence
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score, precision_score, recall_score


from json_loader import JSONLoader
from vectorizer import W2VVectorizer, ELMoVectorizer, BERTVectorizer
from data import DATA_SET_DIR, MODELS_DIR
from configuration import Configuration
from metrics import mean_recall_k, mean_precision_k, mean_ndcg_score, mean_rprecision_k
from neural_networks.lmtc_networks.document_classification import DocumentClassification
from neural_networks.lmtc_networks.label_driven_classification import LabelDrivenClassification


Using TensorFlow backend.


In [2]:
Configuration.configure()

In [3]:
import lmtc

In [4]:
lmtc=lmtc.LMTC()

Load labels' data
-------------------
100%|██████████| 12000/12000 [00:08<00:00, 1420.12it/s]
Labels shape:    4271
Frequent labels: 739
Few labels:      3369
Zero labels:     163


In [11]:
network = LabelDrivenClassification(lmtc.label_terms_ids)

In [13]:
lmtc.label_terms_ids

array([[11820,     0,     0, ...,     0,     0,     0],
       [  485,   310,     0, ...,     0,     0,     0],
       [  485,   324,  3908, ...,     0,     0,     0],
       ...,
       [  167,   625,   697, ...,     0,     0,     0],
       [18721,     0,     0, ...,     0,     0,     0],
       [  538,   477,    94, ...,     0,     0,     0]], dtype=int32)

In [5]:
import pickle

with open("/mnt/localdata/geng/model/lmtc_models/clf_model_predictions/prediction.pickle", "rb") as f:
    pred_labels,true_labels=pickle.load(f)

In [8]:
pred_labels=np.array(pred_labels)

In [14]:
test_documents = lmtc.load_dataset('test')


100%|██████████| 6000/6000 [04:50<00:00, 20.64it/s]


In [16]:
limit = len(test_documents) % Configuration['model']['batch_size'] if Configuration['model']['architecture'] == 'BERT' else 0
test_samples, test_tags = lmtc.process_dataset(test_documents if not limit else test_documents[:-limit])
test_samples, test_targets = lmtc.encode_dataset(test_samples, test_tags)

In [ ]:
network.compile(n_hidden_layers=Configuration['model']['n_hidden_layers'],
                hidden_units_size=Configuration['model']['hidden_units_size'],
                dropout_rate=Configuration['model']['dropout_rate'],
                word_dropout_rate=Configuration['model']['word_dropout_rate'],
                lr=Configuration['model']['lr'])


In [ ]:
predictions=lmtc.predict(network,test_samples)

In [9]:
lmtc.calculate_performance(true_labels,pred_labels)

Overall
----------------------------------------------------


TypeError: list indices must be integers or slices, not tuple

In [13]:
documents = lmtc.load_dataset('test')

AttributeError: module 'lmtc' has no attribute 'load_dataset'

In [15]:
len(documents)

6000

In [19]:
lmtc.load_label_descriptions()

Load labels' data
-------------------
100%|██████████| 12000/12000 [00:09<00:00, 1242.48it/s]
Labels shape:    4271
Frequent labels: 739
Few labels:      3369
Zero labels:     163


In [21]:
test=lmtc.load_dataset("test")

100%|██████████| 6000/6000 [05:21<00:00, 18.69it/s]


In [ ]:
DATA_SET_DIR="/mnt/localdata/geng/lib/lmtc-eurlex57k/data/datasets/"

MODELS_DIR="/mnt/localdata/geng/lib/lmtc-eurlex57k/data/models/"